## DATA PREPROCESSING

In [47]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix

In [85]:
df = pd.read_excel("category.xlsx",sheet_name= [1,4])
loc_to_cat = df[1]
loc_to_cat['category1'] = loc_to_cat['category1'].str.lower()
allUserData = df[4]
#userData = [df[3], df[4], df[5], df[6], df[7]]

#for i in userData[1:]:
#    allUserData = allUserData.append(i)
allUserData.head()

,user_id,weekday,date,start_time,end_date,end_time,duration,category,category_id,location_name,location_category,my_place,location_id,place_id
0,2,2.0,2020-06-29,16:00:00,2020-06-29,19:00:00,03:00:00,meeting,13.0,路易莎,cafe,NaN,4.0,NaN
1,2,2.0,2020-06-29,19:00:00,2020-06-29,20:00:00,01:00:00,food,1.0,全哥鵝肉小炒,restaurant,NaN,5.0,NaN
2,2,2.0,2020-06-29,20:00:00,2020-06-29,20:30:00,00:30:00,leisure,10.0,NaN,NaN,home,NaN,1.0
3,2,2.0,2020-06-29,20:30:00,2020-06-29,21:30:00,01:00:00,shopping,12.0,Miramar,Shopping mall,NaN,6.0,NaN
4,2,2.0,2020-06-29,21:30:00,2020-06-29,23:30:00,02:00:00,study,5.0,NaN,NaN,home,NaN,1.0


In [49]:
allUserData['location']= allUserData['location_id'].combine_first(allUserData['place_id'])
allUserData['location'].fillna("99", inplace = True)
allUserData = allUserData.drop(['date','location_name','location_category','my_place','location_id','place_id'], 1)

In [50]:
def to_mins(ts):
    a = ts.hour*60 + ts.minute
    return a

allUserData['start_time'] = allUserData['start_time'].apply(to_mins)
allUserData['duration'] = allUserData['duration'].apply(to_mins)

allUserData.tail()

,user_id,weekday,start_time,duration,category,category_id,location
300,2,7,120,420,sleep,3,1
301,2,7,540,20,home,2,1
302,2,7,560,40,commute,17,99
303,2,7,600,180,hangout,11,4
304,2,7,780,60,food,1,5


In [51]:
cols = allUserData.columns
for col in cols:
    print(col)

user_id
weekday
start_time
duration
category
category_id
location


## Random Forest Model

In [52]:
train_df = allUserData[allUserData['weekday'] < 5]
test_df = allUserData[allUserData['weekday'] >= 5]
print("Number of traing data is :",len(train_df))
print("Number of testing data is :",len(test_df))

Number of traing data is : 166
Number of testing data is : 139


In [53]:
# without diff
X_train = train_df.iloc[:, np.r_[0:3, 6]].values
y_train = train_df.iloc[:, 5].values

X_test = test_df.iloc[:, np.r_[0:3, 6]].values
y_test = test_df.iloc[:, 5].values

In [54]:
clf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=50, verbose=0,
            warm_start=False)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=50, verbose=0, warm_start=False)

In [55]:
clf.score(X_test, y_test)

0.7841726618705036

In [56]:
imp = clf.feature_importances_
imp

array([0.        , 0.04307985, 0.29493397, 0.66198618])

In [57]:
predictions = clf.predict(X_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           1       0.56      0.78      0.65        18
           2       1.00      0.92      0.96        24
           3       0.86      1.00      0.92        12
           4       0.50      0.75      0.60         4
           5       0.00      0.00      0.00         1
           6       0.69      0.90      0.78        10
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         2
           9       0.25      1.00      0.40         2
          11       0.00      0.00      0.00         3
          12       0.00      0.00      0.00         4
          13       0.00      0.00      0.00         4
          16       0.00      0.00      0.00         3
          17       0.94      0.98      0.96        48
          18       0.00      0.00      0.00         3

   micro avg       0.78      0.78      0.78       139
   macro avg       0.32      0.42      0.35       139
weighted avg       0.71   

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [58]:
print(confusion_matrix(y_test,predictions))

[[14  0  0  2  0  0  0  0  1  0  0  0  0  1  0]
 [ 0 22  2  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 12  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  3  0  0  0  0  0  0  0  0  0  1  0]
 [ 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  9  0  0  1  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0  0  1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  2  0  0  0  0  0  0]
 [ 3  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 2  0  0  0  0  0  0  1  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  4  0  0  0  0  0  0  0  0  0]
 [ 3  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  1  0  0  0  0 47  0]
 [ 1  0  0  0  0  0  0  0  2  0  0  0  0  0  0]]


In [59]:
predictions

array([ 3,  3,  2, 17,  6,  1,  6, 17,  6, 17,  2,  3,  2, 17,  6,  1,  6,
       17,  1,  4, 17, 17,  2,  3,  2, 17,  1,  1, 17,  4,  1,  9,  9, 17,
        2,  3, 17,  6,  1, 17,  1, 17,  6, 17,  2, 17, 17,  9,  9, 17,  2,
        3,  2, 17,  1, 17,  1, 17,  6, 17, 17,  4, 17,  3,  3,  2, 17,  1,
       17,  2, 17,  1, 17,  9,  2,  3,  2, 17,  6, 17,  6, 17,  1, 17,  2,
       17,  9, 17,  2,  3,  2, 17,  1, 17,  1, 17,  1, 17,  1, 17,  4,  4,
       17,  9,  3,  1, 17,  1,  1,  1, 17,  8, 17,  9,  3,  2, 17,  6, 17,
        2, 17,  6,  1, 17,  2,  3,  2, 17,  6, 17,  1,  4, 17,  2,  3,  2,
       17,  1,  1])

## User Data

In [107]:
location = pd.read_csv("0825.csv")
location.tail()

FileNotFoundError: [Errno 2] File b'0825.csv' does not exist: b'0825.csv'

In [108]:
stopLoc = location.drop(['category2','category3','category4','category5'], 1)

def string_to_mins(ts):
    a = int(ts.split(':')[0])
    b = int(ts.split(':')[1])
    c = a*60 + b
    return c

#stopLoc['start_time'] = pd.to_datetime(stopLoc['start_time'], format='%H:%M', errors='ignore').apply(string_to_mins)
stopLoc['longitude'] = round(stopLoc['longitude'],3)
stopLoc['latitude'] = round(stopLoc['latitude'],3)
stopLoc

,location_id,longitude,latitude,start_date,start_time,weekday,duration,speed,name1,name2,name3,name4,name5,category1
0,1,121.576,24.986,2020/07/22,11:00,4,NaN,NaN,myhome,myhome1,myhome2,myhome3,myhome4,c1
1,2,121.443,24.433,2020/07/23,12:00,5,NaN,NaN,apple,apple1,apple2,apple3,apple4,c1
2,3,121.123,24.765,2020/07/24,13:00\n,6,NaN,NaN,banana,banana1,banana2,banana3,banana4,c1
3,4,121.563,25.087,2020-08-24,21:48,2,0.0,-1.000000,軟訊資訊有限公司,阿華食堂,Family Mart,7-Eleven,全聯福利中心Pxmart 內湖文湖,electronics_store
4,5,121.563,25.087,2020-08-24,21:48,2,0.0,-1.000000,軟訊資訊有限公司,阿華食堂,Family Mart,7-Eleven,全聯福利中心Pxmart 內湖文湖,electronics_store
5,6,121.563,25.087,2020-08-24,21:48,2,0.0,-1.000000,軟訊資訊有限公司,阿華食堂,Family Mart,7-Eleven,全聯福利中心Pxmart 內湖文湖,electronics_store
6,7,121.563,25.087,2020-08-24,22:14,2,0.0,0.534089,軟訊資訊有限公司,Family Mart,阿華食堂,7-Eleven,Takming University of Science and Technology,electronics_store
7,8,121.563,25.087,2020-08-24,22:14,2,0.0,0.534089,軟訊資訊有限公司,Family Mart,阿華食堂,7-Eleven,Takming University of Science and Technology,electronics_store
8,9,121.563,25.088,2020-08-24,22:55,2,0.0,-1.000000,Takming University of Science and Technology,7-Eleven,Family Mart,全聯福利中心Pxmart 內湖文湖,赫里翁 - 接待中心,university
9,10,121.563,25.087,2020-08-24,23:52,2,0.0,-1.000000,Family Mart,阿華食堂,軟訊資訊有限公司,7-Eleven,小鳥不吃肉,convenience_store


In [109]:
#delete trivial data
dropList = []

for row in range(1,len(stopLoc)-1):   
    
    #compared with the last record, over 2 place name duplicated will be deleted
    res = set(stopLoc.iloc[row,8:13]) & set(stopLoc.iloc[row-1,8:13])
    if len(res) > 1: 
        dropList.append(row)
    
    #reset duration
    #stopLoc.iloc[row,6] = stopLoc.iloc[row+1,4] - stopLoc.iloc[row,4] 
    
    #duration less than 10, kill
    #if stopLoc.iloc[row,6] < 5:
    #    dropList.append(row)

stopLoc = stopLoc.drop(dropList)
stopLoc = stopLoc.reset_index(drop = True)
stopLoc

,location_id,longitude,latitude,start_date,start_time,weekday,duration,speed,name1,name2,name3,name4,name5,category1
0,1,121.576,24.986,2020/07/22,11:00,4,NaN,NaN,myhome,myhome1,myhome2,myhome3,myhome4,c1
1,2,121.443,24.433,2020/07/23,12:00,5,NaN,NaN,apple,apple1,apple2,apple3,apple4,c1
2,3,121.123,24.765,2020/07/24,13:00\n,6,NaN,NaN,banana,banana1,banana2,banana3,banana4,c1
3,4,121.563,25.087,2020-08-24,21:48,2,0.0,-1.000000,軟訊資訊有限公司,阿華食堂,Family Mart,7-Eleven,全聯福利中心Pxmart 內湖文湖,electronics_store
4,17,121.562,25.086,2020-08-25,10:14,3,0.0,1.025177,DUCATI,TOYOTA 內湖營業所,興沛動物醫院,新生命資訊服務股份有限公司,北都汽車股份有限公司內湖服務廠,point_of_interest
5,18,25.087,25.087,2020-08-25,10:15,3,0.0,1.025177,mo的家,2,NaN,NaN,NaN,School
6,19,121.562,25.085,2020-08-25,10:15,3,0.0,1.063896,來來豆漿,頤園素食小館,MediaTek Inc.,7-ELEVEN,歐客佬咖啡農場內湖店,restaurant
7,20,121.561,25.085,2020-08-25,10:28,3,0.0,7.221824,永祥西點麵包店,Taiwan love rabbit Association (love rabbit home),聖隆五金、水電材料行,來來豆漿,犇鱻涮涮鍋,bakery
8,21,121.560,25.085,2020-08-25,10:28,3,0.0,9.193168,Martial Armour Dazhi,CPC,Wen Hu Elementary School,永豐金證券,聖隆五金、水電材料行,gym
9,22,121.558,25.085,2020-08-25,10:28,3,0.0,9.174859,Miramar,Sym,Grand Victoria Hotel Taipei,永澤園藝社,N°168 Prime Steakhouse 牛排館,transit_station


In [110]:
#calculate duration
from datetime import datetime
dropList = []
a = 10
for i in range(2):
    for row in range(0,len(stopLoc)-1): 

        #reset duration
        #stopLoc.iloc[row,6] = stopLoc.iloc[row+1,4] - stopLoc.iloc[row,4] 

        #duration less than 10, kill
        #if stopLoc.iloc[row,6] < 10:
        #    dropList.append(row)

        #calculate duration
        if stopLoc.iloc[row+1,3] == stopLoc.iloc[row,3]:
            stopLoc.iloc[row,6] = datetime.strptime(stopLoc.iloc[row+1,4], '%H:%M') - datetime.strptime(stopLoc.iloc[row,4], '%H:%M')   
        #else:
        #    stopLoc.iloc[row,6] = datetime.strptime(stopLoc.iloc[row+1,4], '%H:%M') + datetime.strptime('24:00') - datetime.strptime(stopLoc.iloc[row,4], '%H:%M')

        if pd.Timedelta(stopLoc.iloc[row,6]) < pd.Timedelta('00:10:00'):
            dropList.append(row)
            
    stopLoc = stopLoc.drop(dropList)
    stopLoc = stopLoc.reset_index(drop = True)
    a = a + 2
    print(dropList)
    dropList = []

stopLoc = stopLoc.drop(['start_date','name2','name3','name4','name5','speed','longitude','latitude'], 1)
stopLoc

[3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 36, 37, 38, 39, 40]
[]


,location_id,start_time,weekday,duration,name1,category1
0,1,11:00,4,NaN,myhome,c1
1,2,12:00,5,NaN,apple,c1
2,3,13:00\n,6,NaN,banana,c1
3,19,10:15,3,0:41:00,來來豆漿,restaurant
4,81,10:56,3,0:15:00,Teluk Tomimi,natural_feature
5,85,11:11,3,1:46:00,PX MART Daan Yongkang Store,supermarket
6,86,12:57,3,0:35:00,Sun Merry,bakery
7,93,13:32,3,0,喝什麼Ka Pi,cafe


## Map Category

In [106]:
stopLoc = pd.merge(stopLoc, loc_to_cat, how='left', on='category1' )
stopLoc = stopLoc.drop(['name1','category1','location_id'],1)
stopLoc['user_id'] = 2
stopLoc = stopLoc[['user_id','weekday','start_time','duration','location']]
stopLoc
#把start_date加回來
#name1,category1拿掉 place_cat對表換成數字

,start_time,weekday,duration,location,user_id
0,11:00,4,NaN,NaN,2
1,12:00,5,NaN,NaN,2
2,13:00\n,6,NaN,NaN,2
3,10:15,3,0:41:00,5.0,2
4,10:56,3,0:15:00,NaN,2
5,11:11,3,1:46:00,6.0,2
6,12:57,3,0:35:00,4.0,2
7,13:32,3,0,4.0,2


In [ ]:
traing = stopLoc.iloc[:, np.r_[0:4]].values
predictions = clf.predict(traing)

## Myplace

In [379]:
savedplace = pd.read_csv("savedPlace.csv")
savedplace['place_longitude'] = round(saveplace['place_longitude'],3)
savedplace['place_latitude'] = round(saveplace['place_latitude'],3)
#savedplace = savedplace.loc[savedplace['my_place'] == 1]
savedplace = savedplace.drop(['place_name','my_place','place_id'],1)
savedplace = savedplace.rename(columns={"place_longitude": "longitude", "place_latitude": "latitude"})
savedplace = savedplace.drop_duplicates(subset=['longitude', 'latitude'], keep='last')
savedplace.head()

,place_category,longitude,latitude
0,university,121.576,24.986
2,NaN,121.516,25.045


In [380]:
stopLoc = pd.merge(stopLoc, savedplace, how='left', on=['longitude','latitude'])
stopLoc

,location_id,longitude,latitude,start_time,weekday,duration,name1,category1,place_category
0,4,121.516,25.045,930,5,89.0,Notch 咖啡工場 站前店,cafe,NaN
1,5,121.516,25.045,1019,5,0.0,Notch 咖啡工場 站前店,cafe,NaN
